# Day 7: Handy Haversacks

https://adventofcode.com/2020/day/7

## Part 1

In [7]:
# dull bronze bags contain 2 striped indigo bags, 4 plaid black bags, 3 clear violet bags, 1 dull chartreuse bag.

import re

# TODO need a method for matching a content type to the main BagRule?

class BagContent:
    contents_pattern = re.compile(r"(?P<count>\d+)\s(?P<type>.*?)\sbags?\.?")
    no_contents_str = "no other bags."

    def __init__(self, content):
        self._content = content.strip()
        self.empty = False
        self.count = 0
        self.bag_type = None
        if self._content == self.no_contents_str:
            self.empty = True
        else:
            match = self.contents_pattern.match(self._content)
            if not match:
                raise ValueError("Bad match for content %s" % content)
            data = match.groupdict()
            self.count = int(data['count'])
            self.bag_type = data['type']
    
    def __str__(self) -> str:
        if self.empty:
            return "--No contents--"
        plural = "" if self.count == 1 else "s"
        return f"{self.count} bag{plural} of type `{self.bag_type}`"

class BagRule:
    def __init__(self, rule):
        self._full_line = rule
        self.bag_type, self.rule = (x.strip() for x in self._full_line.split('bags contain'))
        self.contents = [BagContent(x) for x in self.rule.split(',')]
        self.bag_types_can_contain = [x.bag_type for x in self.contents if not x.empty]

In [8]:
# Sanity tests for the classes

example = "dull bronze bags contain 2 striped indigo bags, 4 plaid black bags, 3 clear violet bags, 1 dull chartreuse bag."
thing = BagRule(example)
print(f"Type: {thing.bag_type}")
print("May contain:")
for content in thing.contents:
    print(f"  {content}")

# If output looks good, let's proceed.

Type: dull bronze
May contain:
  2 bags of type `striped indigo`
  4 bags of type `plaid black`
  3 bags of type `clear violet`
  1 bag of type `dull chartreuse`


In [19]:
from pathlib import Path

INPUTS = Path('input.txt').resolve().read_text().strip()
RULES = [BagRule(x) for x in INPUTS.split('\n')]
TARGET = 'shiny gold'

# We'll figure out the potential containers of one bag by recursively finding bags
# that are able to contain each type. In each recursion, a list is returned
# and appended (via `.extend`) to the existing set of containers.
# The base case for an empty container returns the empty list, `[]`.
# Finally, the list is run through `set` to remove dupes, then returned again
# as a list.
def get_containers(bag_type):
    containers = [x.bag_type for x in RULES if bag_type in x.bag_types_can_contain]
    if not containers:
        return []
    final_containers = containers[:]
    for container in containers:
        final_containers.extend(get_containers(container))
    return list(set(final_containers))

# This allows us to pick out all containers of our target, the shiny gold bag.
shiny_gold_containers = get_containers(TARGET)

# And the length of that returned list is our answer:
print(f"Shiny gold bags can be eventually be contained in one of {len(shiny_gold_containers)} bag types.")

Shiny gold bags can be eventually be contained in one of 246 bag types.


## Part 2

In [30]:
# For this one, we can again solve by recursion, but in the opposite direction.
# The class I wrote for Part 1 can probably remain unchanged. This time, though,
# I need to compile the number of bags being held, 

# First I did a sanity check, ensuring all the rules are unique:
sanity_check = len(set([x.bag_type for x in RULES])) == len(RULES)
print(f"Sanity check: Are all rules unique for containing bag type? -- {sanity_check}\n")

# That being settled, we can be certain that when we look for a bag_type in the rules,
# we'll only find one target rule. Thus, the [0] index below.
def get_num_bag_contents(bag_type):
    target_rule = [x for x in RULES if x.bag_type == bag_type][0]
    # With the rule in hand, we can iterate through the contents and
    # recursively pick out bag counts for each of those.
    total_count = 0
    for bag in target_rule.contents:
        if bag.empty:
            continue
        # For a non-empty bag, add the count of those contained bags
        # AND the number of bags that bag may contain MULTIPLIED by
        # how many bags there are.
        # So if this bag can contain 2 of a bag type that is found to individually
        # contain 5 other bags, that's (2 + (5 * 2)) = 12
        total_count += bag.count + (get_num_bag_contents(bag.bag_type) * bag.count)
    return total_count

contents = get_num_bag_contents(TARGET)
print(f"'{TARGET}' bags must contain {contents} individual bags.")
print("That's a lot of bags. I better pick different luggage next time.")

Sanity check: Are all rules unique for containing bag type? -- True

'shiny gold' bags must contain 2976 individual bags.
That's a lot of bags. I better pick different luggage next time.
